In [34]:
import pandas as pd
import numpy as np
import ast
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Graph Model - No Updates

## Elective Graph ML

#### Model - GraphSAGE

After some further research by the Seattle Kraken's analytics department, the team has decided to try to implement some graph machine learning into their methods to characterize NHL players. One model of interest is the GraphSAGE model that works to embed player features into float vectors. Using player nodes and the PLAYS_WITH edge, the GraphSAGE will transform player attributes into a useable format for a ML classification model. The model will be trained based of player degree which has been calculated based on how my PLAYS_WITH edges a certain player has. Following training, the model will return the player and their associated embedding vector. 

#### Code

CALL gds.graph.create('Player_sage', <br>
&emsp;&emsp;&emsp;&emsp;                   'Player', {PLAYS_WITH: { <br>
&emsp;&emsp;&emsp;&emsp;                          type: 'PLAYS_WITH', <br>
&emsp;&emsp;&emsp;&emsp;                          orientation: 'UNDIRECTED'}})

CALL gds.degree.mutate('Player_sage', {mutateProperty: 'degree'}) <br>
YIELD nodePropertiesWritten


CALL gds.beta.graphSage.train( <br>
&emsp;&emsp;  'Player_sage', <br>
&emsp;  { <br>
&emsp;&emsp;    modelName: 'graphSage', <br>
&emsp;&emsp;    featureProperties: ['degree'] <br>
&emsp;&emsp;    learningRate:0.001, <br>
&emsp;&emsp;    epochs:10 <br>
&emsp;  } <br>
)

CALL gds.beta.graphSage.stream( <br>
&emsp;&emsp;  'Player_sage', <br>
&emsp;  { <br>
&emsp;&emsp;    modelName: 'graphSage' <br>
&emsp;  }) <br>
YIELD nodeId, embedding <br>
RETURN gds.util.asNode(nodeId).first_name + ' ' + gds.util.asNode(nodeId).last_name AS Player, embedding <br>

In [2]:
sage_results = pd.read_csv('player_embeddings.csv')
sage_results.head()

,Player,embedding
0,Shayne Gostisbehere,"[0.11226033574093451,0.13159826734459273,0.122..."
1,Claude Giroux,"[0.11221969980490319,0.1318266041815291,0.1232..."
2,Travis Konecny,"[0.11223483076995378,0.13173648205579874,0.123..."
3,Ivan Provorov,"[0.11223948957578837,0.1317080497596955,0.1230..."
4,Brandon Manning,"[0.11225547090125365,0.13161756503640212,0.122..."


In [4]:
#the embedding vectors are being read in as a string representation of a list. Apply ast.literal_eval to convert to list
sage_results['embedding'] = sage_results['embedding'].apply(ast.literal_eval)

In [5]:
#expand each embedding element into its own column
embeddings = sage_results['embedding'].apply(pd.Series)
embeddings = embeddings.rename(columns = lambda x: 'embedding_' + str(x))

In [6]:
#concatentate the embedding columns with the original player names
concat_sage = pd.concat([sage_results['Player'], embeddings],axis=1)
concat_sage.head()

,Player,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_54,embedding_55,embedding_56,embedding_57,embedding_58,embedding_59,embedding_60,embedding_61,embedding_62,embedding_63
0,Shayne Gostisbehere,0.112260,0.131598,0.122953,0.148596,0.097817,0.088059,0.119645,0.148200,0.109250,...,0.150990,0.095985,0.161893,0.095068,0.153072,0.186881,0.133661,0.083386,0.175816,0.093023
1,Claude Giroux,0.112220,0.131827,0.123224,0.148910,0.097673,0.087653,0.119305,0.148010,0.108965,...,0.150739,0.095582,0.161601,0.094364,0.152951,0.187348,0.134062,0.083049,0.176651,0.092888
2,Travis Konecny,0.112235,0.131736,0.123127,0.148793,0.097722,0.087796,0.119430,0.148084,0.109067,...,0.150830,0.095729,0.161717,0.094628,0.153002,0.187180,0.133922,0.083164,0.176339,0.092935
3,Ivan Provorov,0.112239,0.131708,0.123098,0.148756,0.097737,0.087840,0.119468,0.148107,0.109097,...,0.150857,0.095775,0.161754,0.094709,0.153018,0.187128,0.133879,0.083198,0.176243,0.092949
4,Brandon Manning,0.112255,0.131618,0.122989,0.148631,0.097795,0.088003,0.119603,0.148182,0.109213,...,0.150959,0.095935,0.161869,0.094990,0.153065,0.186941,0.133717,0.083334,0.175909,0.093003


#### Classification model - let's see if we can predict the player's goals based on their embeddings

In [20]:
stats = pd.read_csv('first_300_game_stats.csv', index_col=0)
stats.head()

,game_id,firstName,lastName,goals
25937,2019020001,Dylan,DeMelo,0
25101,2019020001,Mark,Borowiecki,0
1233,2019020001,Artem,Anisimov,0
37697,2019020001,Scott,Sabourin,1
7788,2019020001,Morgan,Rielly,0


In [21]:
stats['Player'] = stats['firstName'] + ' ' + stats['lastName']
stats = stats[['Player', 'goals']].copy()

In [22]:
#group by player to count the number of goals they scored in the dataset
season_stats = stats.groupby('Player').sum()['goals'].reset_index()
season_stats.sort_values('goals', ascending=False)

,Player,goals
187,David Pastrnak,17
408,Leon Draisaitl,16
62,Auston Matthews,14
165,Connor McDavid,14
22,Alex Ovechkin,14
...,...,...
280,Jacob Middleton,0
561,Paul Carey,0
271,J.C. Beaudin,0
268,Isac Lundestrom,0


#### For simplicity sake, I am converting number of goals into three categories: 10 goals and up = 3, 6-10 goals = 2, 5 and below = 1

In [23]:
season_stats['goals'] = np.where(season_stats['goals'] >= 10, 3, (np.where(season_stats['goals'] <= 5, 1, 2)))
season_stats.sort_values('goals', ascending=False)

,Player,goals
22,Alex Ovechkin,3
88,Brayden Schenn,3
165,Connor McDavid,3
463,Matthew Tkachuk,3
187,David Pastrnak,3
...,...,...
267,Ilya Mikheyev,1
268,Isac Lundestrom,1
269,Ivan Barbashev,1
270,Ivan Provorov,1


In [24]:
sage_players_merge = season_stats.merge(concat_sage, on='Player')
sage_players_merge.head()

,Player,goals,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_54,embedding_55,embedding_56,embedding_57,embedding_58,embedding_59,embedding_60,embedding_61,embedding_62,embedding_63
0,Aaron Ekblad,1,0.112265,0.131563,0.122925,0.148558,0.097828,0.088098,0.119682,0.148227,...,0.151018,0.096030,0.161937,0.095156,0.153093,0.186831,0.133623,0.083413,0.175712,0.093035
1,Aaron Ness,1,0.112290,0.131449,0.122742,0.148369,0.097936,0.088375,0.119899,0.148328,...,0.151179,0.096287,0.162081,0.095576,0.153144,0.186522,0.133343,0.083663,0.175210,0.093131
2,Adam Boqvist,1,0.112257,0.131609,0.122982,0.148622,0.097798,0.088013,0.119612,0.148189,...,0.150965,0.095946,0.161880,0.095012,0.153070,0.186928,0.133708,0.083341,0.175883,0.093006
3,Adam Erne,1,0.112265,0.131574,0.122919,0.148560,0.097836,0.088109,0.119685,0.148220,...,0.151020,0.096033,0.161923,0.095149,0.153084,0.186824,0.133611,0.083430,0.175720,0.093040
4,Adam Fox,1,0.112259,0.131603,0.122962,0.148606,0.097811,0.088045,0.119632,0.148195,...,0.150983,0.095971,0.161886,0.095047,0.153070,0.186896,0.133675,0.083373,0.175840,0.093017


In [30]:
sage_players_merge['goals'].value_counts()

1    624
2     89
3     20
Name: goals, dtype: int64

In [25]:
X = sage_players_merge.drop(columns=['Player', 'goals'])
y = sage_players_merge['goals']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [31]:
parameters = {'kernel': ('linear', 'poly', 'rbf'), 'C':(1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.01), 'degree':(5,4,3,2,1)}

svc = SVC(class_weight='balanced')

grid = GridSearchCV(svc, parameters)

grid.fit(X_train, y_train)

GridSearchCV(estimator=SVC(class_weight='balanced'),
             param_grid={'C': (1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.01),
                         'degree': (5, 4, 3, 2, 1),
                         'kernel': ('linear', 'poly', 'rbf')})

In [32]:
grid.best_params_

{'C': 0.2, 'degree': 5, 'kernel': 'poly'}

In [33]:
grid_pred = grid.predict(X_test)

confusion_matrix(y_test, grid_pred)

array([[83, 86, 37],
       [ 9,  8, 11],
       [ 2,  4,  2]], dtype=int64)

In [36]:
print(classification_report(y_test, grid_pred, target_names=['class 1', 'class 2', 'class 3']))

              precision    recall  f1-score   support

     class 1       0.88      0.40      0.55       206
     class 2       0.08      0.29      0.13        28
     class 3       0.04      0.25      0.07         8

    accuracy                           0.38       242
   macro avg       0.33      0.31      0.25       242
weighted avg       0.76      0.38      0.49       242



#### As you can see the model worked ok. The dataset suffers from an imbalanced target variable, but the SVC class_weight parameter helps to mitigate that effect. The model was best at classifying the first class, players that have not scored many goals. This is expected as this class is the majority of the players. Further optimization is needed to more accurately classify the other classes. 


#### This notebook is a great example of how to use player embeddings as feature inputs into a classification model. An optimized model would be valuable in providing the ability to predict roughly how many goals a player will score. Goals provide direct value to a team

## Cypher Action 1

#### Search Phrase

"Which player have been involved in most goals and assists plays with Player $last_name"

#### Description

Find other plays who are connected to the player of interest when the play event was either a goal or an assist

#### Code 

MATCH (p1:Player {last_name:$last_name})-[i1:INVOLVES]-(pl:Play)-[i2:INVOLVES]-(p2:Player) <br>
WHERE i1.event IN ['Goal','Assist'] <br>
WITH p1,i1, pl,i2,p2 ,count(i1.event) AS degree <br>
RETURN p1, i1, pl,i2,p2 <br>
ORDER BY degree DESC <br>

#### Importance

Goals and assists are the most important aspects to a hockey game. Goals win games. Therefore, it is important to know who is invovled in the most plays in which a goal or assist occured. This cypher action allows a user to input a players last name and determine which players were also involved in that play. This gives the user an idea of who that player plays well with and thus who they have chemistry with. Chemistry among plays is a very important part of hockey as players that do not play well together will not score goals. 

## Cypher Action 2

#### Search Phrase

"Which players are in the community $community"

#### Description

Find the players that are similar to each other by visualizing a community via community ID lookup

#### Code

MATCH (p1:Player)-[r:PLAYS_WITH]-(p2:Player) <br>
WHERE p1.community = \\$community AND p2.community = \\$community <br>
RETURN p1, p2

#### Importance

Using the modularity optimization results previously calculated, this cypher action allows the user to identify players who are in the same community. If the team has found a player that they deemed skilled, this action allows them to search for players who are similar via community ID. Good players are likely associated with other good players

## Visualization 1

#### Top 10 Players by PageRank and their connection to other players, ranked by degree of connections

Cypher Query: <br>

MATCH (p:Player) WHERE EXISTS (p.pagerank) <br>
WITH p ORDER BY p.pagerank DESC LIMIT 10 <br>
CALL apoc.cypher.run(" <br>
&emsp;&emsp;    MATCH (p)<-[r:PLAYS_WITH]-(p2) <br>
&emsp;&emsp;    WITH p,p2,r, SIZE((p)<-[r:PLAYS_WITH]-(p2)) as plays_with_degree <br>
&emsp;&emsp;    RETURN p,r, p2 <br>
&emsp;&emsp;    ORDER BY plays_with_degree DESC <br>
&emsp;&emsp;    LIMIT 10", {p:p}) YIELD value <br>
RETURN value

<table><tr>
<td> <img src="viz1_overall_structure.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="viz1_zoomed.png" alt="Drawing" style="width: 1000px;"/> </td>
</tr></table>

Above are two screenshots of visualization 1, left being a view of the overall structure while right is a zoomed in figure showing example nodes and connections. The overall structure shows there are two main clusters from this query. This is a monopartite graph showing player to player relationship based off the PLAYS_WITH edge. There are two features encoded into this graph. 1). Size of node is proportional to PageRank score, higher score = larger node 2.) Color signifies player position: Light Yellow = Center, Magenta = Right Wing, Cyan = Left Wing, and Orange = Defensemen. 
<br> <br>
There is a lot of useful information in this graph. Here there is a visual representation of who the important players are, based off their PageRank scores. Extending outward, this graph helps to identify who these important players play with. This knowledge allows insight into who plays well whom if they're on the same team or plays well against if they're on another team. This knowledge can then be further broken down by player position.

## Visualization 2

Example use of Cypher Action 1: Which player have been involved in most goals and assists plays with Player Giroux

<img src='viz2.png'>

Visualization 2 is an example of using a dynamic bloom search phrase to investigate which players are directly connected to Claude Giroux in when the associating play was a goal or an assist. Below we can see that players like Jakub Voracek and Ivan Provorov are highly connected to Claude Giroux across many goal/assist plays. This unveils who Giroux plays well with as well as implicitly identifying who he does not. 